In [50]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import seaborn as sns

In [51]:
year = 2021
url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)

In [52]:
page = requests.get(url)
soup = BeautifulSoup(page.text,'html.parser')

In [53]:
soup.findAll('tr',limit=2)
headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
headers = headers[1:]

In [54]:
rows = soup.findAll('tr')[1:]
player_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
stats = pd.DataFrame(player_stats,columns = headers)

In [55]:
rowindex = ['Age','G','GS','MP','FG','FGA','FG%','3P','3PA','3P%','2P','2PA','2P%','eFG%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS']
age, G, GS, MP, FG, FGA, FGper, s3P, s3PA, s3Pper, s2P, s2PA, s2Pper, eFGper, FT, FTA, FTper, ORB, DRB,TRB, AST, STL, BLK, TOV,PF,PTS = 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB','AST','STL','BLK','TOV','PF','PTS'

In [56]:
for i in rowindex:
    stats[i] = stats[i].apply(pd.to_numeric)

In [57]:
stats.fillna(value=0)    

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Precious Achiuwa,PF,21.0,MIA,61.0,4.0,12.1,2.0,3.7,0.544,...,0.509,1.2,2.2,3.4,0.5,0.3,0.5,0.7,1.5,5.0
1,Jaylen Adams,PG,24.0,MIL,7.0,0.0,2.6,0.1,1.1,0.125,...,0.000,0.0,0.4,0.4,0.3,0.0,0.0,0.0,0.1,0.3
2,Steven Adams,C,27.0,NOP,58.0,58.0,27.7,3.3,5.3,0.614,...,0.444,3.7,5.2,8.9,1.9,0.9,0.7,1.3,1.9,7.6
3,Bam Adebayo,C,23.0,MIA,64.0,64.0,33.5,7.1,12.5,0.570,...,0.799,2.2,6.7,9.0,5.4,1.2,1.0,2.6,2.3,18.7
4,LaMarcus Aldridge,C,35.0,TOT,26.0,23.0,25.9,5.4,11.4,0.473,...,0.872,0.7,3.8,4.5,1.9,0.4,1.1,1.0,1.8,13.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,Delon Wright,PG,28.0,SAC,27.0,8.0,25.8,3.9,8.3,0.462,...,0.833,1.0,2.9,3.9,3.6,1.6,0.4,1.3,1.1,10.0
727,Thaddeus Young,PF,32.0,CHI,68.0,23.0,24.3,5.4,9.7,0.559,...,0.628,2.5,3.8,6.2,4.3,1.1,0.6,2.0,2.2,12.1
728,Trae Young,PG,22.0,ATL,63.0,63.0,33.7,7.7,17.7,0.438,...,0.886,0.6,3.3,3.9,9.4,0.8,0.2,4.1,1.8,25.3
729,Cody Zeller,C,28.0,CHO,48.0,21.0,20.9,3.8,6.8,0.559,...,0.714,2.5,4.4,6.8,1.8,0.6,0.4,1.1,2.5,9.4


In [58]:
#The Hollinger Game Score stat: PTS + 0.4 * FG - 0.7 * FGA - 0.4*(FTA - FT) + 0.7 * ORB + 0.3 * DRB + STL + 0.7 * AST + 0.7 * BLK - 0.4 * PF - TOV.
hollpts,hollfg,hollfga,hollfta,hollft,hollorb,holldrb,hollstl,hollast,hollblk,hollpf,holltov = [],[],[],[],[],[],[],[],[],[],[],[]

def get_holl(pts,fg,fga,fta,ft,orb,drb,stl,ast,blk,pf,tov):
    hollingerEQ = lambda PTS,FG,FGA,FTA,FT,ORB,DRB,STL,AST,BLK,PF,TOV: PTS + 0.4 * FG - 0.7 * FGA - 0.4*(FTA - FT) + 0.7 * ORB + 0.3 * DRB + STL + 0.7 * AST + 0.7 * BLK - 0.4 * PF - TOV
    return hollingerEQ(pts,fg,fga,fta,ft,orb,drb,stl,ast,blk,pf,tov)
    # for i in range(677):
    #     hollpts.append(stats[PTS].loc[i])



In [59]:
GameScore = []
for i in range(len(stats)):
    hollpts.append(stats['PTS'].iloc[i])
    hollfg.append(stats['FG'].iloc[i])
    hollfga.append(stats['FGA'].iloc[i])
    hollfta.append(stats['FTA'].iloc[i])
    hollft.append(stats['FT'].iloc[i])
    hollorb.append(stats['ORB'].iloc[i])
    holldrb.append(stats['DRB'].iloc[i])
    hollstl.append(stats['STL'].iloc[i])
    hollast.append(stats['AST'].iloc[i])
    hollblk.append(stats['BLK'].iloc[i])
    hollpf.append(stats['PF'].iloc[i])
    holltov.append(stats['TOV'].iloc[i])

for i in range(len(stats)):
    GameScore.append(round(get_holl(hollpts[i],hollfg[i],hollfga[i],hollfta[i],hollft[i],hollorb[i],holldrb[i],hollstl[i],hollast[i],hollblk[i],hollpf[i],holltov[i]),5))

In [60]:
stats['Game Score'] = GameScore

In [61]:
stats.head(1)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Game Score
0,Precious Achiuwa,PF,21.0,MIA,61.0,4.0,12.1,2.0,3.7,0.544,...,1.2,2.2,3.4,0.5,0.3,0.5,0.7,1.5,5.0,4.05


In [64]:
stats.set_index('Player',inplace=True)

In [65]:
stats.nlargest(10,['Game Score'])

,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Game Score
Player,,,,,,,,,,,,,,,,,,,,,
Nikola Jokić,C,25.0,DEN,72.0,72.0,34.6,10.2,18.0,0.566,1.3,...,2.8,8.0,10.8,8.3,1.3,0.7,3.1,2.7,26.4,25.38
Giannis Antetokounmpo,PF,26.0,MIL,61.0,61.0,33.0,10.3,18.0,0.569,1.1,...,1.6,9.4,11.0,5.9,1.2,1.2,3.4,2.8,28.1,24.01
Stephen Curry,PG,32.0,GSW,63.0,63.0,34.2,10.4,21.7,0.482,5.3,...,0.5,5.0,5.5,5.8,1.2,0.1,3.4,1.9,32.0,23.75
Joel Embiid,C,26.0,PHI,51.0,51.0,31.1,9.0,17.6,0.513,1.1,...,2.2,8.4,10.6,2.8,1.0,1.4,3.1,2.4,28.5,23.12
James Harden,PG,31.0,BRK,36.0,35.0,36.6,7.8,16.6,0.471,2.7,...,0.8,7.7,8.5,10.9,1.3,0.8,4.0,2.4,24.6,23.10
James Harden,PG-SG,31.0,TOT,44.0,43.0,36.6,7.8,16.7,0.466,2.8,...,0.8,7.1,7.9,10.8,1.2,0.8,4.0,2.3,24.6,22.72
Damian Lillard,PG,30.0,POR,67.0,67.0,35.8,9.0,19.9,0.451,4.1,...,0.5,3.7,4.2,7.5,0.9,0.3,3.0,1.5,28.8,22.49
Kevin Durant,PF,32.0,BRK,35.0,32.0,33.1,9.3,17.2,0.537,2.4,...,0.4,6.7,7.1,5.6,0.7,1.3,3.4,2.0,26.9,21.88
Bradley Beal,SG,27.0,WAS,60.0,60.0,35.8,11.2,23.0,0.485,2.2,...,1.2,3.5,4.7,4.4,1.2,0.4,3.1,2.3,31.3,21.75


In [66]:
teams = []
for i in stats['Tm']:
    if i not in teams and i != None:
        teams.append(i)


In [67]:
teams.sort()

In [68]:
teams

['ATL',
 'BOS',
 'BRK',
 'CHI',
 'CHO',
 'CLE',
 'DAL',
 'DEN',
 'DET',
 'GSW',
 'HOU',
 'IND',
 'LAC',
 'LAL',
 'MEM',
 'MIA',
 'MIL',
 'MIN',
 'NOP',
 'NYK',
 'OKC',
 'ORL',
 'PHI',
 'PHO',
 'POR',
 'SAC',
 'SAS',
 'TOR',
 'TOT',
 'UTA',
 'WAS']